## Importing the Modules

In [1]:
import numpy as np
import pandas as pd
import os
import time as tm
import cv2
import matplotlib.pyplot as plt
import seaborn as sns


Datasets

In [2]:
utk_folder = "C:\\Users\\mayan\\Desktop\\CODES\\Null Codes\\Long Hair Detection\\UTKFace"
celeb_folder = "C:\\Users\\mayan\\Desktop\\CODES\\Null Codes\\Long Hair Detection\\CelebA Dataset"
attributes = "C:\\Users\\mayan\\Desktop\\CODES\\Null Codes\\Long Hair Detection\\list_attr_celeba.csv"

Pre-Processing the celeb images

In [3]:
def preprocess_image(img_path):
    image = cv2.imread(img_path)
    image = cv2.resize(image, (48,48))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image / 255.0
    return image

start = tm.time()
celeb_images = []
for photo in os.listdir(celeb_folder):
    image_path = os.path.join(celeb_folder, photo)
    img = preprocess_image(image_path)
    celeb_images.append(img)
print(f"Time taken for image pre-processing of the CelebA Dataset: {int((tm.time() - start)/60)} minutes")

Time taken for image pre-processing of the CelebA Dataset: 1 minutes


Pre-Processing the UTKFace images

In [4]:

def preprocessing_image(img_path):
    image = cv2.imread(img_path)
    image = cv2.resize(image, (48,48))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image
utk_images = []
for photo in os.listdir(utk_folder):
    image_path = os.path.join(utk_folder, photo)
    image = preprocessing_image(image_path)
    utk_images.append(image)
print("UTKFace Pre-Processing Completed!")

UTKFace Pre-Processing Completed!


Extracting the ages from the UTK images

In [5]:
utk_ages = []

for photo in os.listdir(utk_folder):
    age = int(photo.split('_')[0])
    utk_ages.append(age)

In [6]:
utk_images = np.array(utk_images)
utk_ages = np.array(utk_ages)
print(utk_images.shape)
print(utk_ages.shape)

(23708, 48, 48, 3)
(23708,)


In [7]:
print(len(utk_images) == len(utk_ages))

True


In [8]:
utk_gender = []  # 0 for Male, 1 for Female

for photo in os.listdir(utk_folder):
    gender = int(photo.split('_')[1])
    utk_gender.append(gender)

utk_gender = np.array(utk_gender)


In [9]:
print(utk_gender)
len(utk_gender)

[0 0 1 ... 1 1 1]


23708

## Model for age detection

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D 
from tensorflow.keras.optimizers import Adam, RMSprop

model = Sequential()
# First layer
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(48,48,3)))
model.add(MaxPooling2D((2,2)))

# Second Layer
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

# Third layer
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

# Fourth Layer
model.add(Conv2D(256, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))

# Flattening the CNN
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(1, activation='linear'))

# Compiling and producing the summary of the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_14 (Conv2D)              │ (None, 46, 46, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 2, 2, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 421,441 (1.61 MB)

 Trainable params: 421,441 (1.61 MB)

 Non-trainable params: 0 (0.00 B)

Spiliting the dataset into Training and Testing

In [34]:
from sklearn.model_selection import train_test_split as tts 
from tensorflow.keras.callbacks import EarlyStopping

X_train, X_test, y_train, y_test = tts(utk_images, utk_ages, test_size=0.2, random_state=42)
early_stopping = EarlyStopping(monitor='val_mae', patience=5, restore_best_weights=True)

model.fit(X_train, y_train, epochs = 100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])


Epoch 1/100
593/593 ━━━━━━━━━━━━━━━━━━━━ 17s 26ms/step - loss: 507.5836 - mae: 17.1620 - val_loss: 210.6288 - val_mae: 10.9880
Epoch 2/100
593/593 ━━━━━━━━━━━━━━━━━━━━ 15s 25ms/step - loss: 238.5294 - mae: 11.5990 - val_loss: 157.6260 - val_mae: 9.0753
Epoch 3/100
593/593 ━━━━━━━━━━━━━━━━━━━━ 16s 26ms/step - loss: 197.4331 - mae: 10.4308 - val_loss: 134.1773 - val_mae: 8.6037
Epoch 4/100
593/593 ━━━━━━━━━━━━━━━━━━━━ 16s 27ms/step - loss: 170.5962 - mae: 9.6273 - val_loss: 125.6771 - val_mae: 8.2982
Epoch 5/100
593/593 ━━━━━━━━━━━━━━━━━━━━ 16s 26ms/step - loss: 151.6304 - mae: 8.9863 - val_loss: 128.7622 - val_mae: 8.4293
Epoch 6/100
593/593 ━━━━━━━━━━━━━━━━━━━━ 16s 27ms/step - loss: 138.6267 - mae: 8.5957 - val_loss: 120.1549 - val_mae: 7.8082
Epoch 7/100
593/593 ━━━━━━━━━━━━━━━━━━━━ 16s 27ms/step - loss: 131.6582 - mae: 8.3511 - val_loss: 112.7887 - val_mae: 7.5691
Epoch 8/100
593/593 ━━━━━━━━━━━━━━━━━━━━ 17s 29ms/step - loss: 134.8175 - mae: 8.4566 - val_loss: 127.6533 - val_mae: 8.2

In [35]:
model.evaluate(X_test, y_test)

149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 108.6424 - mae: 7.4412


[112.7886962890625, 7.56908655166626]

In [36]:
model.save('Age_Detector.h5')

In [14]:
pred_utk_ages = model.predict(X_test)

149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


In [15]:
print(pred_utk_ages)

[[61.33904 ]
 [17.13849 ]
 [27.909698]
 ...
 [19.361115]
 [55.67939 ]
 [ 9.483886]]


Evaluating the Model

In [16]:
utk_ages = utk_ages.reshape(-1,1)
pred_utk_ages = pred_utk_ages.reshape(-1,1).astype(int)

from sklearn.metrics import r2_score,accuracy_score, mean_absolute_error, mean_squared_error

r2 = r2_score(y_test, pred_utk_ages)
print(f"R2 Score: {r2}")

mae = mean_absolute_error(y_test, pred_utk_ages)
print(f"MAE: {mae}")


R2 Score: 0.7095468622358665
MAE: 7.502952340784479


Saving the Model

## Extracting attributes from the excel sheet

Making a Gender Model

In [17]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Initialize the Sequential model
gen_model = models.Sequential()

# Add the first Convolutional Block
gen_model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)))
gen_model.add(layers.MaxPooling2D((2, 2)))

# Add the second Convolutional Block
gen_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
gen_model.add(layers.MaxPooling2D((2, 2)))

# Add the third Convolutional Block
gen_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
gen_model.add(layers.MaxPooling2D((2, 2)))

# Flatten the output from the convolutional layers
gen_model.add(layers.Flatten())

# Add a fully connected (dense) layer
gen_model.add(layers.Dense(128, activation='relu'))

# Add the output layer for binary classification
gen_model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
gen_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Display the model architecture
gen_model.summary()


c:\Users\mayan\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 46, 46, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 355,649 (1.36 MB)

 Trainable params: 355,649 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
print(utk_images.shape)
utk_gender.shape

(23708, 48, 48, 3)


(23708,)

In [19]:
utk_gender = np.array(utk_gender)
utk_images = np.array(utk_images)

X_train, X_test, y_train, y_test = tts(utk_images, utk_gender, test_size=0.2)

gen_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 14s 21ms/step - accuracy: 0.7160 - loss: 1.8816 - val_accuracy: 0.8180 - val_loss: 0.3987
Epoch 2/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - accuracy: 0.8552 - loss: 0.3364 - val_accuracy: 0.8543 - val_loss: 0.3335
Epoch 3/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 13s 22ms/step - accuracy: 0.8689 - loss: 0.2910 - val_accuracy: 0.8802 - val_loss: 0.2842
Epoch 4/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 12s 21ms/step - accuracy: 0.8849 - loss: 0.2642 - val_accuracy: 0.8724 - val_loss: 0.3086
Epoch 5/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - accuracy: 0.8963 - loss: 0.2436 - val_accuracy: 0.8739 - val_loss: 0.2953
Epoch 6/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - accuracy: 0.8998 - loss: 0.2371 - val_accuracy: 0.8804 - val_loss: 0.2790
Epoch 7/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - accuracy: 0.9078 - loss: 0.2150 - val_accuracy: 0.8806 - val_loss: 0.2817
Epoch 8/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - accuracy: 0.9085 - loss: 0.2113 - 

In [20]:
gen_model.evaluate(X_test,y_test)

149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8917 - loss: 0.2711


[0.2685732841491699, 0.8924504518508911]

In [21]:
pred_utk_gender = gen_model.predict(X_test).reshape(-1)
# pred_utk_gender = (gen_model.predict(X_test) > 0.5).astype(int)
print(pred_utk_gender)

149/149 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
[0.08182237 0.9998955  0.99999493 ... 0.6234163  0.99997497 0.99693984]


In [22]:
gen_model.save('Gender_Detector.h5')

## Hair Classification

1. Man,
  2. bald or receding hair line ------> short
  3. not bald and not receding and wavy hair -----> long



4. Woman, 
   long hair


In [23]:
celeb_df = pd.read_csv(attributes)

In [24]:
celeb_df = celeb_df[["Bald","Male","Receding_Hairline","Bangs","Wavy_Hair"]]

In [25]:
hairClass = [] # 0 for Short Hair,  1 for Long Hair  

for index, row in celeb_df.iterrows():
    if row['Male']:   # if its a man
        if (row['Bald']==1 or row['Receding_Hairline']==1) or row['Wavy_Hair']==-1:
            hairClass.append(0)
            continue
        if (row['Wavy_Hair']==1):
            hairClass.append(1)
            continue
    else: # if its a woman
        if row["Bangs"]==1:
            hairClass.append(0)
            continue
        else:
            hairClass.append(1)
            continue




In [26]:
hairClass = np.array(hairClass)
print(len(hairClass))
print(len(celeb_images))

50000
50000


In [27]:
# Define the model
hair_model = Sequential()

# Add layers
hair_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 3)))
hair_model.add(MaxPooling2D(pool_size=(2, 2)))

hair_model.add(Conv2D(64, (3, 3), activation='relu'))
hair_model.add(MaxPooling2D(pool_size=(2, 2)))

hair_model.add(Conv2D(128, (3, 3), activation='relu'))
hair_model.add(MaxPooling2D(pool_size=(2, 2)))

hair_model.add(Flatten())
hair_model.add(Dense(128, activation='relu'))
hair_model.add(Dropout(0.5))
hair_model.add(Dense(1, activation='sigmoid'))

hair_model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
hair_model.summary()


c:\Users\mayan\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)               │ (None, 46, 46, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 355,649 (1.36 MB)

 Trainable params: 355,649 (1.36 MB)

 Non-trainable params: 0 (0.00 B)

Making the Hair Model

In [28]:
celeb_images = np.array(celeb_images)
hairClass = np.array(hairClass)

X_train, X_test, y_train, y_test = tts(celeb_images, hairClass, test_size=0.2, random_state=42)

hair_model.fit(X_train, y_train, epochs = 10, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 25s 37ms/step - accuracy: 0.7100 - loss: 0.5785 - val_accuracy: 0.7596 - val_loss: 0.4801
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - accuracy: 0.7626 - loss: 0.4841 - val_accuracy: 0.7701 - val_loss: 0.4690
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 22s 35ms/step - accuracy: 0.7894 - loss: 0.4464 - val_accuracy: 0.7900 - val_loss: 0.4399
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 22s 35ms/step - accuracy: 0.8010 - loss: 0.4296 - val_accuracy: 0.7806 - val_loss: 0.4525
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 22s 35ms/step - accuracy: 0.8079 - loss: 0.4115 - val_accuracy: 0.7979 - val_loss: 0.4302
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 22s 35ms/step - accuracy: 0.8200 - loss: 0.3921 - val_accuracy: 0.8067 - val_loss: 0.4278
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - accuracy: 0.8258 - loss: 0.3802 - val_accuracy: 0.8018 - val_loss: 0.4548
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 34ms/step - accuracy: 0.8370 - loss: 0.3607 - 

In [29]:
hair_model.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7744 - loss: 0.4913


[0.49913397431373596, 0.7712000012397766]

In [30]:
pred_celeb_hair = hair_model.predict(X_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


In [31]:
hair_model.save('Hair_Classifier.h5')